In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install datasets
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
import transformers
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise
from tqdm.auto import tqdm

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("SpartanCinder/GPT2-finetuned-lyric-generation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

You are using a model of type gpt2 to instantiate a model of type gptj. This is not supported for all configurations of models and can yield errors.


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
import random
from datasets import Dataset, DatasetDict

def randomize_lines(data_path):
    with open(data_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        print(len(lines))
        random.shuffle(lines)
        return lines

train_data_path = '/content/MyDrive/MyDrive/NLP Project/train_couplets.txt'
test_data_path = '/content/MyDrive/MyDrive/NLP Project/test_couplets.txt'

train_lines = randomize_lines(train_data_path)
test_lines = randomize_lines(test_data_path)

train_dataset = Dataset.from_dict({"text": train_lines})
test_dataset = Dataset.from_dict({"text": test_lines})


# Concatenate train and test datasets into a single dataset
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

tokenizer = transformers.AutoTokenizer.from_pretrained("SpartanCinder/GPT2-finetuned-lyric-generation")

def tokenize_function(examples):
  return tokenizer(examples["text"])

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns="text")

del tokenizer
del dataset

832
8


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/832 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
block_size = 128
checkpoint_num = 8000

def group_texts(examples):
  # Concatenate all texts.
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  #print(concatenated_examples)
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  #print(f"Total length: {total_length}")
  total_length = (total_length // block_size) * block_size
  # Split by chunks of max_len.
  result = {
    k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
    for k, t in concatenated_examples.items()
  }
  result["labels"] = result["input_ids"].copy()
  return result

verses_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=500,
    num_proc=8,
)

Map (num_proc=8):   0%|          | 0/832 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("SpartanCinder/GPT2-finetuned-lyric-generation")
gpt = model

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

training_args = TrainingArguments(
    output_dir="/content/MyDrive/MyDrive/NLP Project/", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=100, # number of training epochs
    # evaluation_strategy="steps",
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    # # eval_steps = 50, # Number of update steps between two evaluations.
    # # save_steps = 2000, # after # steps model is saved
    # save_total_limit = 999,# limits total # of checkpoints
    # load_best_model_at_end = True, # save best checkpoint after training complete
    # warmup_steps = 50,# number of warmup steps for learning rate scheduler
    # # prediction_loss_only=True,
    # save_strategy="steps",  #changed from "steps" to "no" to try to use callback to save entire model
    weight_decay=0.001,
    # resume_from_checkpoint=f"/content-{checkpoint_num}")
)

trainer = Trainer(
    model=gpt,
    args=training_args,
    train_dataset=verses_datasets['train'],
    eval_dataset=verses_datasets['test'])
    #callbacks=[SaveCallback])

trainer.train()
# print(trainer)

Step,Training Loss
500,1.485400
1000,0.673000
1500,0.367000
2000,0.245200
2500,0.172500
3000,0.119000
3500,0.083800
4000,0.065200
4500,0.054600
5000,0.046300


TrainOutput(global_step=9000, training_loss=0.19892561764187283, metrics={'train_runtime': 1356.727, 'train_samples_per_second': 26.461, 'train_steps_per_second': 6.634, 'total_flos': 2345095987200000.0, 'train_loss': 0.19892561764187283, 'epoch': 100.0})

In [ ]:
torch.save(gpt, "/content/MyDrive/MyDrive/NLP Project/rap_lyrics_model.pt")

In [ ]:
import re
def generate_verse():
    prompt = "And that ass feel like jello (jello)"
    device = torch.cuda.current_device()

    print("...getting prediction...")
    tokenizer = transformers.AutoTokenizer.from_pretrained("SpartanCinder/GPT2-finetuned-lyric-generation")
    model = model
    with torch.no_grad():
        result_length = 75
        prompt = "~ " + prompt + " =1G->2G= "
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        beam_outputs = model.generate(inputs["input_ids"],
            max_length=result_length,
            top_k=50, top_p=0.95,
            do_sample=True, temperature=0.7, pad_token_id=50256,
            num_return_sequences=10)

        lines = []
        for beam in beam_outputs:
            text = tokenizer.decode(beam, skip_special_tokens=True)
            line = text.split(" =1G->2G= ")[1]
            line = line[:line.find(" ~")]
            if line not in lines:
                lines.append(line.strip("'").strip('"'))

        filtered_lines = [replace_n_word(line) for line in lines] #remove n-word
        # filtered_lines = [line for line in filtered_lines if has_characters(line)]

        if len(filtered_lines) == 0:
            return generate_verse()  # Recursively retry if no suitable lines found

        # Delete unnecessary variables to conserve memory
        del inputs, beam_outputs, lines

        return filtered_lines

def replace_n_word(line):
    # Define a regular expression pattern to match n-word and hyphenated cases
    pattern = r'\b\w*-?nigga\w*\b'
    # Use re.search to find the pattern in the line
    line = re.sub(pattern,"*****", line, flags=re.IGNORECASE)
    # returns line with stars if present or w no modifications if word not found
    return line

def has_characters(self,line):
# Strip removes leading and trailing whitespaces including tabs and newlines
     return False if line.strip() == "" else True


In [ ]:
x = generate_verse()

...getting prediction...


In [ ]:
print(x)

['ick ih-f y-uw-r hh-ax|v-ih-n g-er-l p-r-aa|b-l-ax-m-z ay f-iy-l b-ae-d f-a', '~~ yeah, yeah (yeah, yeah, yeah, yeah, yeah, yeah)', '~~ four, four, four, four, fou', 'ick-er|k-ax-n ax-n ax w-iy|k-eh-n-d l-ay-k y-uw|zh-ax|w-ax-l ]\n[ ih-f ay w-aa', 'ips b-l-ow ax s-t-ae-k f-ao-r y-ao-r n-ih|g-ae-s w-ih-dh y-ao-r t-r-ae|p-ax-ng ', 'igg|l-ih|t-ax-n p-r-aa|b-l-ax-m', '~~ yeah i know, you just got cash (mm, bust it)', "~~ (just awaken shaken once again, ho, you know it's on)", '~~ b-ey|b-iy w-eh-n w-iy g-aa-n s-l-ay-d']


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("SpartanCinder/GPT2-finetuned-lyric-generation")
input_ids = tokenizer.encode("i'm gonna hit the bars nobody ever can, i shall be honest like no other man", return_tensors='pt')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the input_ids tensor to the same device as the model
input_ids = input_ids.to(device)
output = model.generate(input_ids, max_length=400, num_return_sequences=5, do_sample=True, top_p=0.9)
final_verse = tokenizer.decode(output[0], skip_special_tokens=True)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


i'm gonna hit the bars nobody ever can, i shall be honest like no other man ]
[ to my lover, i'd never lie =2G->2P= t-ax m-ay l-ah|v-er ay-d n-eh|v-er l-ay ]
[ ay w-aa-n|d-er-d t-ax l-aa =1P->1G= i wandered to la ]
 la la la =1G->1P= l-aa l-aa l-aa l-aa l-aa ]
[ dh-ax hh-ay-z ae-n-d dh-ax l-ow-z =1P->1G= the highs and the lows ]
[ y-uw hh-ah-n|d-r-ax hh-ay-z dh-ax hh-aw-s-t m-ae-n =1P->1G= you know, i've never felt like this before (oh, oh) ]
[ iy|v-ax-n ax-f ay l-ay-k ax f-ah-k dh-ax l-ow-z dh-ax hh-ay-z dh-ax hh-ow-s-t m-ae-n ~
[ ih-t-s aa-n dh-ax w-er-l ax|b-aw-t iy|v-ax-n ax-f ay l-ay-k hh-ay-z dh-ax l-ow-z dh-ax hh-ay-z dh-ax hh-ay-z dh-ax l-ow-z =1P->1G= the highs and the lows ]
[ y-uw l-ah-v y-uw hh-ah-n


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("SpartanCinder/GPT2-finetuned-lyric-generation")
input_ids = tokenizer.encode("from concrete jungles to stars above, this beats the canvas, lets paint with blood", return_tensors='pt')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the input_ids tensor to the same device as the model
input_ids = input_ids.to(device)
output = model.generate(input_ids, max_length=400, num_return_sequences=5, do_sample=True, top_p=0.9)
final_verse = tokenizer.decode(output[0], skip_special_tokens=True)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


from concrete jungles to stars above, this beats the canvas, lets paint with blood =2G->2P= b-ih-ch d-ow-n-t g-eh-t t-uw k-ah-m|f-er|t-ax|b-ax-l ]
[ k-r-ao-s m-ay hh-aa-r-t hh-ow-p t-ax d-ay =1P->1G= cross my heart, hope to die ]
[ hh-aw m-ah-ch b-eh|t-er k-ae-n ay sh-ow m-ay l-ah-v f-ao-r y-uw =2P->2G= hate investigatin', that sh- was a waste, man ]
[ ah-p ih-f ay hh-uw t-ax s-ah-m|th-ax-ng m-iy =1P->1G= i believe that if i fly, i'll prolly end up somewhere in paradise ]
[ eh|v-r-iy|th-ih-n-d t-ax l-ah-v f-ao-r y-uw ~
[ eh|v-r-iy|th-ih-n-d t-ax l-ah-v f-ao-r y-uw =1P->1G= eh|v-r-iy|th-ih-ng w-eh-n ay sh-ow m-ay l-ah-v f-ao-r y-uw ~
[ ah-p ih-f ay hh-uw t-ax s-ah-m|th-ax-ng m-iy ~
[ y-uw =1P->1G= yeah|v-r-


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("SpartanCinder/GPT2-finetuned-lyric-generation")
input_ids = tokenizer.encode("Mom's spaghetti knees weak arms are heavy", return_tensors='pt')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the input_ids tensor to the same device as the model
input_ids = input_ids.to(device)
output = model.generate(input_ids, max_length=400, num_return_sequences=5, do_sample=True, top_p=0.9)
final_verse = tokenizer.decode(output[0], skip_special_tokens=True)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Mom's spaghetti knees weak arms are heavy with me ]
~ if i hit it two times, then i like her =1G->2G= i hit it two times, then i like her ~
[ ay-m f-iy|l-ih-n s-t-ah-k f-iy|l-ax-n s-t-ah-k =1P->1G= i'm feelin' stuck (feelin' stuck) ]
[ e'rybody lookin' for a dance floor to run on =2G->2P= eh|er|ax|b-aa|d-iy l-uh|k-ax-n f-ao-r ax d-ae-n-s f-l-ao-r t-ax r-ah-n aa-n ]
[ l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa ow =1P->1G= la la la la la, la ]
[ la la la, la =1G->1P= l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa l-aa ow ]
[ la la la la, la =1P->1G= la la la la, la ]
[ la la la la, la ~
[ hh-aa-t m-ay l-ay-t-s aa-n dh-ax m-ih|n-y-uw =1P->1G= uh, uh, uh, uh ]
[ sh-iy n-iy-


In [ ]:
!pip install eng_to_ipa
import eng_to_ipa as ipa

def is_vowel(character):

    ipa_vowels = "aeiou"
    return character in ipa_vowels

def is_whitespace(character):
    return character==' ' or character=='\n'

def get_phonetic_transcription(lyrics):
    lines = lyrics.splitlines()
    phonetic_lines = [ipa.convert(line) for line in lines]
    phonetic = "\n".join(phonetic_lines)

    return phonetic

import re
import numpy as np

def syllable_similarity(bar1, bar2):
    similarity = abs(syllables.estimate(bar1) - syllables.estimate(bar2))
    return similarity

def get_syllable_count_difference(lyrics):
    lines = lyrics.splitlines()
    i = 0
    similarity = 0
    while i < len(lines)-1:
        similarity += syllable_similarity(lines[i], lines[i+1])
        i += 2
    return similarity/len(lines)

def get_rhyme_density(lyrics, lookback=15):
    bars = SongLyrics(lyrics, lookback)
    return bars.avg_rhyme_length

def get_longest_rhyme(lyrics, lookback=15):
    bars = SongLyrics(lyrics, lookback)
    return bars.get_longest_rhyme()[0]

def get_unique_words(lyrics):
    words = lyrics.split()
    unique_words = set(words)
    return len(unique_words)/len(words)

def print_lyrics_stats(lyrics, lookback=15, artist=None, title=None):
    bars = SongLyrics(lyrics, lookback, artist, title)
    print(bars.title)
    print('------------------------------------------')
    print("Average rhyme length: %.3f\n" % bars.avg_rhyme_length)
    bars.print_rhyme(bars.longest_rhyme)
    print("Average syllable count difference: %.3f\n" % get_syllable_count_difference(lyrics))
    print("Percentage unique words: %.3f\n" % get_unique_words(lyrics))



class SongLyrics:
    '''
    This class is used to store and preprocess rap lyrics and calculate
    statistics like average rhyme length out of the lyrics.
    '''

    def __init__(self, text=None, lookback=15, artist=None, title=None):
        '''
        Lyrics can be read from the file (default) or passed directly
        to this constructor.
        '''
        self.raw_text = None
        # How many previous words are checked for a rhyme.
        self.lookback = lookback
        self.raw_text = text
        if artist == None or title == None:
            self.title = "Generated Song"
        else:
            self.title = title + " by " + artist

        if self.raw_text is not None:
            cleaning_ok = self.clean_text(self.raw_text)
            self.compute_vowel_representation()
            self.avg_rhyme_length, self.longest_rhyme = self.calculate_rhyme_stats()

    def clean_text(self, text):
        '''
        Preprocess text by removing unwanted characters and duplicate rows.
        '''

        self.text = text
        # If there are more than 2 consecutive newlines, remove some of them
        # (just to make the cleaned text look prettier)
        self.text = re.sub('\n\n+', '\n\n', self.text)
        # Remove duplicate rows
        self.lines = self.text.split('\n')

        unique_lines = set()
        new_text = ''
        for line in self.lines:
            line = line.strip()
            if len(line) > 0 and line in unique_lines:
                continue
            # Remove lines that are within brackets/parenthesis
            if len(line) >= 2 and ((line[0]=='[' and line[-1]==']') or (line[0]=='(' and line[-1]==')')):
                continue
            unique_lines.add(line)
            new_text += line + '\n'

        self.text = new_text

    def compute_vowel_representation(self):
        '''
        Compute a representation of the lyrics where only vowels are preserved.
        '''
        self.vowels_only = [] # Lyrics with all but vowels removed
        self.vowel_indices = [] # Indices of the vowels in self.text list
        self.word_end_indices = [] # Indices of the last characters of each word
        self.words = [] # List of words in the lyrics
        self.line_indices = []

        self.original_text = self.text
        self.text = get_phonetic_transcription(self.text)
        self.original_word_end_indices = []
        self.original_words = []

        previous_space_index = -1 # Index of the previous space char
        line_index = 0 # Line index of the current character
        # Go through the lyrics char by char
        for i in range(len(self.text)):
            self.line_indices.append(line_index)
            character = self.text[i]
            if is_vowel(character):
                # Ignore double vowels
                if i > 0 and self.text[i-1] == character:
                    # Index of a double vowel points to the latter occurrence
                    self.vowel_indices[-1] = i
                    continue
                self.vowels_only.append(character)
                self.vowel_indices.append(i)
            elif is_whitespace(character):
                if character in '\n':
                    line_index += 1
                elif character in '.!?' and i < len(self.text)-1 and self.text[i+1] != '\n':
                    line_index += 1
                if len(self.vowels_only) > 0 and not is_whitespace(self.text[i-1]):
                    new_word = self.text[previous_space_index+1:self.vowel_indices[-1]+1]
                    no_vowels = True
                    for char in new_word:
                        if is_vowel(char):
                            no_vowels = False
                            break
                    if no_vowels:
                        previous_space_index = i
                        continue
                    self.word_end_indices.append(len(self.vowels_only)-1)
                    self.words.append(new_word)
                previous_space_index = i

        self.original_lines = self.original_text.split('\n')

    def rhyme_length(self, word_position_2):
        '''
        Length of rhyme (in vowels). The latter part of the rhyme ends with
        word self.words[word_position_2].

        Input:
            word_position_2       Word index of the end of the rhyme.
        '''
        max_length = 0
        max_word_position_1 = None
        word_position_1 = max(0, word_position_2 - self.lookback)
        while word_position_1 < word_position_2:
            rhyme_length = self.fixed_rhyme_length(word_position_1, word_position_2)
            if rhyme_length > max_length:
                max_length = rhyme_length
                max_word_position_1 = word_position_1
            word_position_1 += 1
        return max_length, max_word_position_1

    def fixed_rhyme_length(self, word_position_1, word_position_2):
        '''
        Length of rhyme (in vowels). The first part of the rhyme ends with
        self.words[word_position_1] and the latter part with word self.words[word_position_2].

        Input:
            word_position_1       Word index of the last word in the first part of the rhyme.
            word_position_2       Word index of the end of the rhyme.
        '''
        if word_position_1 < 0:
            return 0
        elif self.words[word_position_1] == self.words[word_position_2]:
            return 0
        vowel_index_1 = self.word_end_indices[word_position_1]
        vowel_index_2 = self.word_end_indices[word_position_2]
        length = 0
        while self.vowels_only[vowel_index_1 - length] == self.vowels_only[vowel_index_2 - length]:
            if word_position_1 > 0 and vowel_index_1 - length <= self.word_end_indices[word_position_1 - 1] and word_position_2 > 0 and vowel_index_2 - length <= self.word_end_indices[word_position_2 - 1]:
                prev_start_index_1 = self.vowel_indices[vowel_index_1 - length]
                while prev_start_index_1 > 0 and not is_whitespace(self.text[prev_start_index_1 - 1]):
                    prev_start_index_1 -= 1
                prev_start_index_2 = self.vowel_indices[vowel_index_2 - length]
                while prev_start_index_2 > 0 and not is_whitespace(self.text[prev_start_index_2 - 1]):
                    prev_start_index_2 -= 1
                next_start_index_1 = self.vowel_indices[vowel_index_1 - length]
                while next_start_index_1 < len(self.text) - 1 and not is_whitespace(self.text[next_start_index_1 + 1]):
                    next_start_index_1 += 1
                next_start_index_2 = self.vowel_indices[vowel_index_2 - length]
                while next_start_index_2 < len(self.text) - 1 and not is_whitespace(self.text[next_start_index_2 + 1]):
                    next_start_index_2 += 1
                if next_start_index_1 - prev_start_index_1 == next_start_index_2 - prev_start_index_2 and self.text[prev_start_index_1:next_start_index_1 + 1] == self.text[prev_start_index_2:next_start_index_2 + 1]:
                    break
            length += 1
            if vowel_index_1 - length < 0 or vowel_index_2 - length <= vowel_index_1:
                break
        if length == 1:
            length = 0
        return length

    def calculate_rhyme_stats(self):
        '''
        Compute the average rhyme length of the song and the longest rhyme.

        Output:
            Average rhyme length (float)
            Longest rhyme which is a 3-tuple with:
                (length, word index of the first part of the rhyme,
                         word index of the latter part of the rhyme)
        '''
        rhyme_lengths = []
        max_rhyme = (0, None, None)
        for word_position_2 in range(1, len(self.word_end_indices)):
            rhyme_length, word_position_1 = self.rhyme_length(word_position_2)
            rhyme_lengths.append(rhyme_length)
            if rhyme_length > max_rhyme[0]:
                max_rhyme = (rhyme_length, word_position_1, word_position_2)
        rhyme_lengths = np.array(rhyme_lengths)
        if len(rhyme_lengths) > 0:
            avg_rhyme_length = np.mean(rhyme_lengths)
        else:
            avg_rhyme_length = 0
        return avg_rhyme_length, max_rhyme

    def get_avg_rhyme_length(self):
        return self.avg_rhyme_length

    def print_song_stats(self):
        print('------------------------------------------')
        print("Avg rhyme length: %.3f\n" % self.avg_rhyme_length)

        self.print_rhyme(self.longest_rhyme)

    def print_rhyme(self, rhyme_tuple):
        print(self.get_rhyme_str(rhyme_tuple))

    def get_rhyme_str(self, rhyme_tuple):
        ret = ''
        rhyme_length, word_position_1, word_position_2 = rhyme_tuple
        if word_position_1 is None or word_position_2 is None:
            return ''
        vowel_index_2 = self.vowel_indices[self.word_end_indices[word_position_2]]
        vowel_index_2_original = vowel_index_2
        while not is_whitespace(self.text[vowel_index_2]):
            vowel_index_2 += 1
        vowel_index_1 = self.vowel_indices[self.word_end_indices[word_position_1] - rhyme_length]
        vowel_index_1_original = vowel_index_1
        while self.text[vowel_index_1] != '\n' and vowel_index_1 > 0:
            vowel_index_1 -= 1

        capitalized_line = ''
        rhyming_vowels_1, rhyming_vowels_2 = self.get_rhyming_vowels(rhyme_tuple)
        for i in range(vowel_index_1, vowel_index_2 + 1):
            if i == min(rhyming_vowels_1) or i == min(rhyming_vowels_2):
                capitalized_line += ' | ' + self.text[i]
            elif i == max(rhyming_vowels_1) or i == max(rhyming_vowels_2):
                capitalized_line += self.text[i] + '|'
            else:
                capitalized_line += self.text[i]
        ret += "Longest rhyme (l=%d): %s\n" % (rhyme_length, capitalized_line)
        line_begin = self.line_indices[vowel_index_1]
        line_end = self.line_indices[vowel_index_2]
        for i in range(line_begin, line_end + 1):
            if i < len(self.original_lines):
                ret += self.original_lines[i] + '\n'
        return ret

    def get_longest_rhyme(self):
        rhyme_str = self.get_rhyme_str(self.longest_rhyme)
        return self.longest_rhyme[0], rhyme_str

    def get_rhyming_vowels(self, rhyme_tuple):
        rhyme_length, word_position_1, word_position_2 = rhyme_tuple
        if word_position_1 is None or word_position_2 is None:
            return ([-1], [-1])

        rhyming_vowels_1 = []
        count_caps = 0
        index = self.vowel_indices[self.word_end_indices[word_position_1]]
        while count_caps < rhyme_length:
            if is_vowel(self.text[index]):
                rhyming_vowels_1.append(index)
                if self.text[index] != self.text[index + 1]:
                    count_caps += 1
            index -= 1

        rhyming_vowels_2 = []
        count_caps = 0
        index = self.vowel_indices[self.word_end_indices[word_position_2]]
        last_index = index
        while count_caps < rhyme_length:
            if is_vowel(self.text[index]):
                rhyming_vowels_2.append(index)
                if index == last_index or self.text[index] != self.text[index + 1]:
                    count_caps += 1
            index -= 1

        return (rhyming_vowels_1, rhyming_vowels_2)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eng_to_ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822607 sha256=85b8c448b50ab4f3a363ff327b31a341480e55f62f0bb1bff7c72eb2aa2416fc
  Stored in directory: /root/.cache/pip/wheels/5b/ab/07/fe6722f710d8ef8bd0ccb4eb689ef96f5552f3fc0c80c1aa9c
Successfully built eng_to_ipa


In [ ]:

import numpy as np
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import bigrams, ngrams, everygrams
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends
from nltk.lm import MLE, KneserNeyInterpolated, Lidstone, Laplace, AbsoluteDiscountingInterpolated


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
final_verse = "Thick if your h-aven girl problems I feel bad fa yeah, yeah (yeah, yeah, yeah, yeah, yeah, yeah) four, four, four, four, fou Thicker than awake like yourself If I wa Lips blow at steak for your ni-gaes with your tra-ping Big-lation problem yeah I know, you just got cash (mm, bust it) just awaken shaken once again, ho, you know it's on) Bye bye when we going s-layed"
result_final = re.sub(r'[ ]+\n[ ]+', r'\n', ''.join(final_verse))
print(result_final)


Thick if your h-aven girl problems I feel bad fa yeah, yeah (yeah, yeah, yeah, yeah, yeah, yeah) four, four, four, four, fou Thicker than awake like yourself If I wa Lips blow at steak for your ni-gaes with your tra-ping Big-lation problem yeah I know, you just got cash (mm, bust it) just awaken shaken once again, ho, you know it's on) Bye bye when we going s-layed


In [ ]:
def get_rhyme_density(lyrics, lookback=15):
    bars = SongLyrics(lyrics, lookback)
    return bars.avg_rhyme_length
results_rd = np.array(get_rhyme_density(result_final))

In [ ]:
round(np.mean(results_rd),2)

0.54

In [ ]:
results_lr = np.array(get_longest_rhyme(result_final))

In [ ]:
round(np.mean(results_lr),2)

3.0

In [ ]:
results_uw = np.array(get_unique_words(result_final))

In [ ]:
print(round(np.mean(results_uw),2))

0.8
